In [1]:
import pandas as pd
import datetime
from sqlalchemy import create_engine

In [2]:
data = pd.read_csv("Data/data.csv")
genre_data = pd.read_csv("Data/data_w_genres.csv")
year_data = pd.read_csv("Data/data_by_year.csv")
artist_data = pd.read_csv("Data/data_by_artist.csv")

In [3]:
organized_data = data[["id",
                       "name",
                       "artists",
                       "year",
                       "duration_ms",
                       "explicit",
                       "acousticness",
                       "danceability",
                       "energy",
                       "instrumentalness",
                       "liveness",
                       "loudness",
                       "popularity",
                       "speechiness",
                       "tempo"                     
                      ]]

In [4]:
spotify_data = organized_data.rename(columns={"id": "ID", 
                                              "name": "Song", 
                                              "artists": "Artists", 
                                              "year": "Year",
                                              "duration_ms": "Duration (M:S)",
                                              "explicit": "Explicit",
                                              "acousticness": "Acousticness",
                                              "danceability": "Danceability",
                                              "energy": "Energy",
                                              "instrumentalness": "Instrumentalness",
                                              "liveness": "Liveness",
                                              "loudness": "Loudness", 
                                              "popularity": "Popularity", 
                                              "speechiness": "Speechiness", 
                                              "tempo": "Tempo"})

In [5]:
#convert 'Duration' column in milliseconds to "Minutes:Seconds" format 
millis = spotify_data["Duration (M:S)"]


MS_list=[]
for m in millis:
   
   
    date = datetime.datetime.fromtimestamp(m/1000.0)
    date = date.strftime('%M:%S')
    MS_list.append(date)
    
MS_list   

spotify_data["Duration (M:S)"]=MS_list

In [6]:
#clean the 'Artist' column
artists = spotify_data['Artists']
artists_l = []
for a in artists:
    test1 = a.strip('[]\'\'').replace("'",'')
    artists_l.append(test1)
spotify_data['Artists'] = artists_l

In [7]:
#round the columns to the nearest 2 digit decimal point
spotify_final=spotify_data.round({'Acousticness':2,'Danceability':2,'Energy':2,'Instrumentalness':2,'Liveness':2,'Loudness':2,'Speechiness': 2, 'Tempo': 2})
spotify_final

,ID,Song,Artists,Year,Duration (M:S),Explicit,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Popularity,Speechiness,Tempo
0,6KbQ3uYMLKb5jDxLF7wYDD,Singende Bataillone 1. Teil,Carl Woitschach,1928,02:38,0,1.00,0.71,0.20,0.56,0.15,-12.43,0,0.05,118.47
1,6KuQTIu1KoTTkLXKrwlLPV,"Fantasiestücke, Op. 111: Più tosto lento","Robert Schumann, Vladimir Horowitz",1928,04:42,0,0.99,0.38,0.01,0.90,0.08,-28.45,0,0.05,83.97
2,6L63VW0PibdM1HDSBoqnoM,Chapter 1.18 - Zamek kaniowski,Seweryn Goszczyński,1928,01:44,0,0.60,0.75,0.22,0.00,0.12,-19.92,0,0.93,107.18
3,6M94FkXd15sOAOQYRnWPN8,Bebamos Juntos - Instrumental (Remasterizado),Francisco Canaro,1928,03:00,0,1.00,0.78,0.13,0.89,0.11,-14.73,0,0.09,108.00
4,6N6tiFZ9vLTSOIxkj8qKrd,"Polonaise-Fantaisie in A-Flat Major, Op. 61","Frédéric Chopin, Vladimir Horowitz",1928,11:27,0,0.99,0.21,0.20,0.91,0.10,-16.83,1,0.04,62.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169904,4KppkflX7I3vJQk7urOJaS,Skechers (feat. Tyga) - Remix,"DripReport, Tyga",2020,02:43,1,0.17,0.88,0.44,0.00,0.09,-7.46,75,0.14,100.01
169905,1ehhGlTvjtHo2e4xJFB0SZ,Sweeter (feat. Terrace Martin),"Leon Bridges, Terrace Martin",2020,02:47,0,0.02,0.72,0.38,0.03,0.11,-10.91,64,0.04,128.00
169906,52eycxprLhK3lPcRLbQiVk,How Would I Know,"Kygo, Oh Wonder",2020,03:00,0,0.54,0.51,0.54,0.00,0.11,-9.33,70,0.10,123.70
169907,3wYOGJYD31sLRmBgCvWxa4,I Found You,"Cash Cash, Andy Grammer",2020,02:47,0,0.07,0.65,0.76,0.00,0.22,-2.56,70,0.04,129.92


In [9]:
engine = create_engine('sqlite:///Spotify.db', echo=True)
sqlite_connection = engine.connect()

2020-09-02 21:26:03,518 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-09-02 21:26:03,526 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:03,534 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-09-02 21:26:03,536 INFO sqlalchemy.engine.base.Engine ()


In [10]:
spotify_table = "Spotify_Data"
spotify_final.to_sql(spotify_table, sqlite_connection, if_exists='replace')

2020-09-02 21:26:03,748 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Spotify_Data")
2020-09-02 21:26:03,749 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:03,754 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Spotify_Data")
2020-09-02 21:26:03,759 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:03,762 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-09-02 21:26:03,764 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:03,767 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Spotify_Data")
2020-09-02 21:26:03,770 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:03,778 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Spotify_Data' AND type = 'table'
2020-09-02 21:26:03,818 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:03,821 INFO sqlalchemy.engine.base.Engine PRAGMA main.fore

In [11]:
Danceability=spotify_final[['ID','Song','Artists','Year','Duration (M:S)','Danceability']]
Danceability_order=Danceability.sort_values('Danceability',inplace=False,ascending=False)


In [12]:
spotify_table = "Groovy_Songs"
Danceability_order.to_sql(spotify_table, sqlite_connection, if_exists='replace')


2020-09-02 21:26:18,765 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Groovy_Songs")
2020-09-02 21:26:18,770 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:18,773 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Groovy_Songs")
2020-09-02 21:26:18,775 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:18,788 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-09-02 21:26:18,791 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:18,800 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Groovy_Songs")
2020-09-02 21:26:18,802 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:18,814 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Groovy_Songs' AND type = 'table'
2020-09-02 21:26:18,817 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:18,835 INFO sqlalchemy.engine.base.Engine PRAGMA main.fore

In [13]:
Spa_music0=spotify_final[['ID','Song','Artists','Year','Duration (M:S)','Instrumentalness']]
Spa_music=Spa_music0.sort_values('Instrumentalness',inplace=False,ascending=False)
Spa_music 

,ID,Song,Artists,Year,Duration (M:S),Instrumentalness
15930,4fD4bjo6ZQp8tlF8Kyy6Ad,Rain Sound: Pouring,Rain Sounds XLE Library,2017,02:21,1.0
46972,605JTCltCe1IcqrGXQ294K,Brown Noise for Sleep,Sounds for Life,2009,11:10,1.0
143967,5svhO7n5a0IWF13hqZWaVc,City Rain,Rain Recorders,2017,04:00,1.0
125835,50N6EPhnmazJFkWvzqynhb,Downpoar,Rain Sound Studio,2017,02:32,1.0
23862,2eWqBNH136U8Et0UTxBacB,White Noise - BP 228 hz,Granular,2017,01:44,1.0
...,...,...,...,...,...,...
69857,6wIBZCIbbZh6ACDkWC5TX7,Ah Yeah,"Robert Glasper Experiment, Musiq Soulchild, Ch...",2012,05:08,0.0
69856,6ol1tnuiEhigHqHPwqQxeq,Stoner Girl (feat. Pat Brown),"MOD SUN, Pat Brown",2012,04:09,0.0
69855,4vpNOBwJSl8HA3pLXY6c8U,Druggys Wit Hoes Again (feat. Ab Soul),"ScHoolboy Q, Ab-Soul",2012,03:39,0.0
69854,1zugfOhn6BebboqGcpUlba,We Are Young (Glee Cast Version),Glee Cast,2012,04:09,0.0


In [14]:
spotify_table = "Spa_music"
Spa_music.to_sql(spotify_table, sqlite_connection, if_exists='replace')

2020-09-02 21:26:31,886 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Spa_music")
2020-09-02 21:26:31,887 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:31,890 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Spa_music")
2020-09-02 21:26:31,894 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:31,897 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-09-02 21:26:31,899 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:31,902 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Spa_music")
2020-09-02 21:26:31,925 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:31,931 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Spa_music' AND type = 'table'
2020-09-02 21:26:31,932 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:31,935 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list

In [15]:
artist_data = artist_data[["artists",
                           "acousticness",
                          "danceability",
                          "energy",
                          "instrumentalness",
                          "liveness",
                          "loudness", 
                          "speechiness", 
                          "tempo",
                          "valence",
                          "popularity",
                          "key"]]

In [16]:
artist_data = artist_data.rename(columns={"artists": "Artists",
                                              "acousticness": "Acousticness",
                                              "danceability": "Danceability",
                                              "energy": "Energy",
                                              "instrumentalness": "Instrumentalness",
                                              "liveness": "Liveness",
                                              "loudness": "Loudness", 
                                              "speechiness": "Speechiness", 
                                              "tempo": "Tempo",
                                          "valence": "Valence",
                                          "popularity": "Popularity",
                                          "key": "Key"
                                          })

In [17]:
artist_data.drop

<bound method DataFrame.drop of                                                  Artists  Acousticness  \
0                       "Cats" 1981 Original London Cast      0.575083   
1                              "Cats" 1983 Broadway Cast      0.862538   
2            "Fiddler On The Roof” Motion Picture Chorus      0.856571   
3         "Fiddler On The Roof” Motion Picture Orchestra      0.884926   
4      "Joseph And The Amazing Technicolor Dreamcoat"...      0.605444   
...                                                  ...           ...   
27616                                                鳳飛飛      0.884000   
27617                                                黃品源      0.541000   
27618                                                黃國隆      0.785455   
27619                                                黃蜀娟      0.925143   
27620                                                 黑豹      0.381000   

       Danceability    Energy  Instrumentalness  Liveness   Loudness  \
0      

In [18]:
artist_data = artist_data.round({'Acousticness':2,'Danceability':2,'Energy':2,'Instrumentalness':2,
                                 'Liveness':2,'Loudness':2,'Speechiness': 2, 'Tempo': 2})

In [19]:
Groovy_artists = artist_data.sort_values("Danceability", ascending=False)

In [20]:
spotify_table = "Groovy_Artists"
Groovy_artists.to_sql(spotify_table, sqlite_connection, index=False, if_exists='replace')
Groovy_artists.to_sql()

2020-09-02 21:26:41,823 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Groovy_Artists")
2020-09-02 21:26:41,825 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:41,829 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Groovy_Artists")
2020-09-02 21:26:41,831 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:41,834 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-09-02 21:26:41,836 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:41,838 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Groovy_Artists")
2020-09-02 21:26:41,841 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:41,845 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Groovy_Artists' AND type = 'table'
2020-09-02 21:26:41,855 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:26:41,860 INFO sqlalchemy.engine.base.Engine PRAGMA m

TypeError: to_sql() missing 2 required positional arguments: 'name' and 'con'

In [21]:
Popular_artists = artist_data.sort_values("Popularity", ascending=False)

In [22]:
spotify_table = "Popular_Artists"
Popular_artists.to_sql(spotify_table, sqlite_connection, index=False, if_exists='replace')

2020-09-02 21:27:46,529 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Popular_Artists")
2020-09-02 21:27:46,530 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:27:46,533 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Popular_Artists")
2020-09-02 21:27:46,534 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:27:46,537 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-09-02 21:27:46,538 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:27:46,540 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Popular_Artists")
2020-09-02 21:27:46,541 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:27:46,546 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Popular_Artists' AND type = 'table'
2020-09-02 21:27:46,547 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:27:46,550 INFO sqlalchemy.engine.base.Engine PRAG

In [23]:
year_data

,year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode
0,1921,0.895823,0.425661,229911.914062,0.236784,0.322330,0.215814,-17.095437,0.077258,100.397758,0.425495,0.351562,7,1
1,1922,0.939236,0.480000,167904.541667,0.237026,0.440470,0.238647,-19.179958,0.115419,101.376139,0.534056,0.138889,10,1
2,1923,0.976329,0.568462,178356.301775,0.246936,0.401932,0.236656,-14.373882,0.098619,112.456598,0.624788,5.727811,0,1
3,1924,0.935575,0.548654,188461.649789,0.347033,0.583955,0.237875,-14.202304,0.090210,120.653359,0.668574,0.603376,10,1
4,1925,0.965422,0.571890,184130.699620,0.264373,0.408893,0.243094,-14.516707,0.115457,115.671715,0.616430,2.707224,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2016,0.280290,0.599976,219400.763840,0.592877,0.074646,0.180198,-7.949913,0.107298,119.070344,0.430769,61.371254,0,1
96,2017,0.289916,0.612286,209343.613000,0.586739,0.098209,0.194218,-8.422697,0.111752,116.840277,0.414465,64.861500,1,1
97,2018,0.271941,0.664930,200919.119000,0.590591,0.035948,0.171781,-7.253666,0.128140,122.004325,0.447141,67.276000,1,1
98,2019,0.289298,0.644215,197733.133000,0.578796,0.076518,0.167161,-8.041738,0.124799,118.868163,0.465856,69.655500,1,1


In [24]:
year_data = year_data[['year', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence', 'popularity', 'key']]

In [25]:
year_data = year_data.rename(columns={'year': 'Year', 'acousticness': 'Acousticness', "danceability": "Danceability",
                                              "energy": "Energy",
                                              "instrumentalness": "Instrumentalness",
                                              "liveness": "Liveness",
                                              "loudness": "Loudness", 
                                              "speechiness": "Speechiness", 
                                              "tempo": "Tempo",
                                          "valence": "Valence",
                                          "popularity": "Popularity",
                                          "key": "Key"})

In [26]:
year_data

,Year,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Valence,Popularity,Key
0,1921,0.895823,0.425661,0.236784,0.322330,0.215814,-17.095437,0.077258,100.397758,0.425495,0.351562,7
1,1922,0.939236,0.480000,0.237026,0.440470,0.238647,-19.179958,0.115419,101.376139,0.534056,0.138889,10
2,1923,0.976329,0.568462,0.246936,0.401932,0.236656,-14.373882,0.098619,112.456598,0.624788,5.727811,0
3,1924,0.935575,0.548654,0.347033,0.583955,0.237875,-14.202304,0.090210,120.653359,0.668574,0.603376,10
4,1925,0.965422,0.571890,0.264373,0.408893,0.243094,-14.516707,0.115457,115.671715,0.616430,2.707224,5
...,...,...,...,...,...,...,...,...,...,...,...,...
95,2016,0.280290,0.599976,0.592877,0.074646,0.180198,-7.949913,0.107298,119.070344,0.430769,61.371254,0
96,2017,0.289916,0.612286,0.586739,0.098209,0.194218,-8.422697,0.111752,116.840277,0.414465,64.861500,1
97,2018,0.271941,0.664930,0.590591,0.035948,0.171781,-7.253666,0.128140,122.004325,0.447141,67.276000,1
98,2019,0.289298,0.644215,0.578796,0.076518,0.167161,-8.041738,0.124799,118.868163,0.465856,69.655500,1


In [27]:
Groovy_years = year_data.sort_values("Danceability", ascending=False)

In [28]:
Groovy_years

,Year,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Valence,Popularity,Key
99,2020,0.247374,0.673077,0.611914,0.039052,0.177048,-7.204024,0.143505,121.228704,0.482755,63.111048,1
97,2018,0.271941,0.664930,0.590591,0.035948,0.171781,-7.253666,0.128140,122.004325,0.447141,67.276000,1
6,1927,0.934606,0.649960,0.256159,0.402276,0.167779,-14.726812,0.116197,114.611465,0.660659,0.832215,7
8,1929,0.591111,0.647005,0.240513,0.219461,0.235988,-16.607089,0.485278,111.353988,0.638208,0.327922,7
98,2019,0.289298,0.644215,0.578796,0.076518,0.167161,-8.041738,0.124799,118.868163,0.465856,69.655500,1
...,...,...,...,...,...,...,...,...,...,...,...,...
28,1949,0.907895,0.441623,0.244627,0.293479,0.210266,-15.610530,0.103308,108.876602,0.428594,3.307000,0
27,1948,0.934265,0.439915,0.247542,0.392072,0.221532,-15.512074,0.086625,107.375217,0.443625,1.582441,0
32,1953,0.891139,0.436234,0.265310,0.290270,0.225679,-15.551768,0.096054,109.014178,0.423354,3.177436,0
0,1921,0.895823,0.425661,0.236784,0.322330,0.215814,-17.095437,0.077258,100.397758,0.425495,0.351562,7


In [29]:
spotify_table = "Groovy_Years"
Groovy_years.to_sql(spotify_table, sqlite_connection, index=False, if_exists='replace')

2020-09-02 21:27:50,592 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Groovy_Years")
2020-09-02 21:27:50,596 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:27:50,598 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Groovy_Years")
2020-09-02 21:27:50,601 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:27:50,604 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-09-02 21:27:50,606 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:27:50,608 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Groovy_Years")
2020-09-02 21:27:50,610 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:27:50,614 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Groovy_Years' AND type = 'table'
2020-09-02 21:27:50,616 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:27:50,620 INFO sqlalchemy.engine.base.Engine PRAGMA main.fore

In [30]:
decade_year = year_data.groupby((year_data['Year']//10)*10).mean()

In [31]:
decade_year = decade_year[['Acousticness', 'Danceability', 'Energy', 'Instrumentalness',
       'Liveness', 'Loudness', 'Speechiness', 'Tempo', 'Valence', 'Popularity']]

In [32]:
decade_year = decade_year.round({'Acousticness':2,'Danceability':2,'Energy':2,'Instrumentalness':2,
                                 'Liveness':2,'Loudness':2,'Speechiness': 2, 'Tempo': 2, 'Valence': 2, 'Popularity': 2})

In [33]:
decade_year

,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Valence,Popularity
Year,,,,,,,,,,
1920,0.88,0.55,0.25,0.42,0.22,-16.05,0.17,110.13,0.57,1.54
1930,0.88,0.54,0.29,0.28,0.23,-14.03,0.15,112.69,0.58,2.69
1940,0.88,0.47,0.25,0.38,0.22,-15.32,0.14,107.17,0.49,1.63
1950,0.84,0.48,0.29,0.24,0.21,-14.79,0.10,110.60,0.48,9.93
1960,0.62,0.50,0.42,0.15,0.21,-12.74,0.06,115.47,0.56,25.63
1970,0.39,0.53,0.54,0.11,0.22,-11.39,0.06,120.13,0.59,35.05
1980,0.29,0.55,0.60,0.12,0.21,-11.13,0.06,121.49,0.57,37.00
1990,0.29,0.57,0.59,0.10,0.20,-9.85,0.08,119.33,0.55,43.63
2000,0.26,0.58,0.66,0.08,0.20,-7.37,0.09,121.36,0.53,49.41


In [34]:
spotify_table = "Decades"
decade_year.to_sql(spotify_table, sqlite_connection, if_exists='replace')

2020-09-02 21:27:53,895 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Decades")
2020-09-02 21:27:53,897 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:27:53,899 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Decades")
2020-09-02 21:27:53,901 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:27:53,904 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-09-02 21:27:53,906 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:27:53,908 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Decades")
2020-09-02 21:27:53,911 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:27:53,915 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Decades' AND type = 'table'
2020-09-02 21:27:53,917 INFO sqlalchemy.engine.base.Engine ()
2020-09-02 21:27:53,920 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("Decade